# pyiron

https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

## Define workflow with pyiron_base

In [1]:
from nfdi_ing_workflow import (
    generate_mesh as _generate_mesh, 
    convert_to_xdmf as _convert_to_xdmf,
    poisson as _poisson,
    plot_over_line as _plot_over_line,
    substitute_macros as _substitute_macros,
    compile_paper as _compile_paper,
)

In [2]:
from pyiron_base import Project, job

from python_workflow_definition.pyiron_base import write_workflow_json

In [3]:
generate_mesh = job(_generate_mesh)
convert_to_xdmf = job(_convert_to_xdmf, output_key_lst=["xdmf_file", "h5_file"])
poisson = job(_poisson, output_key_lst=["numdofs", "pvd_file", "vtu_file"])
plot_over_line = job(_plot_over_line)
substitute_macros = job(_substitute_macros)
compile_paper = job(_compile_paper)

In [4]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [5]:
domain_size = 2.0

In [6]:
gmsh_output_file = generate_mesh(
    domain_size=domain_size,
    pyiron_project=pr
)

In [7]:
meshio_output_dict = convert_to_xdmf(
    gmsh_output_file=gmsh_output_file,
    pyiron_project=pr,
)

In [8]:
poisson_dict = poisson(
    meshio_output_xdmf=meshio_output_dict.output.xdmf_file, 
    meshio_output_h5=meshio_output_dict.output.h5_file,
    pyiron_project=pr,
)

In [9]:
pvbatch_output_file = plot_over_line(
    poisson_output_pvd_file=poisson_dict.output.pvd_file, 
    poisson_output_vtu_file=poisson_dict.output.vtu_file,
    pyiron_project=pr,
)

In [10]:
macros_tex_file = substitute_macros( 
    pvbatch_output_file=pvbatch_output_file, 
    ndofs=poisson_dict.output.numdofs, 
    domain_size=domain_size,
    pyiron_project=pr,
)

In [11]:
paper_output = compile_paper(
    macros_tex=macros_tex_file, 
    plot_file=pvbatch_output_file,
    pyiron_project=pr,
)

In [12]:
workflow_json_filename = "pyiron_base_nfdi.json"

In [13]:
write_workflow_json(delayed_object=paper_output, file_name=workflow_json_filename)

## Load Workflow with aiida

In [14]:
from aiida import load_profile

load_profile()

Profile<uuid='3b96612f59bb474b9a4c16a7676ab9bc' name='pwd'>

In [15]:
from python_workflow_definition.aiida import load_workflow_json

In [16]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [17]:
wg.run()

04/04/2025 02:49:20 PM <3075> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGraphEngine|continue_workgraph]: tasks ready to run: generate_mesh6
04/04/2025 02:49:22 PM <3075> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGraphEngine|update_task_state]: Task: generate_mesh6, type: PyFunction, finished.
04/04/2025 02:49:22 PM <3075> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGraphEngine|continue_workgraph]: tasks ready to run: convert_to_xdmf5
04/04/2025 02:49:24 PM <3075> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGraphEngine|update_task_state]: Task: convert_to_xdmf5, type: PyFunction, finished.
04/04/2025 02:49:24 PM <3075> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGraphEngine|continue_workgraph]: tasks ready to run: poisson4
04/04/2025 02:49:28 PM <3075> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGrap

## Load Workflow with jobflow

In [18]:
from python_workflow_definition.jobflow import load_workflow_json

/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [19]:
from jobflow.managers.local import run_locally

In [20]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [21]:
result = run_locally(flow)
result

2025-04-04 14:49:38,125 INFO Started executing jobs locally
2025-04-04 14:49:39,097 INFO Starting job - generate_mesh (96c2a02e-82a1-4ec2-8c8f-03aaaaaa2199)
2025-04-04 14:49:40,737 INFO Finished job - generate_mesh (96c2a02e-82a1-4ec2-8c8f-03aaaaaa2199)
2025-04-04 14:49:40,738 INFO Starting job - convert_to_xdmf (27e8a0e1-b612-4ab4-8dec-f13a2e92c152)
2025-04-04 14:49:42,432 INFO Finished job - convert_to_xdmf (27e8a0e1-b612-4ab4-8dec-f13a2e92c152)
2025-04-04 14:49:42,433 INFO Starting job - poisson (45f843da-fde4-4fc1-8eda-5a6595fa6f53)
2025-04-04 14:49:45,325 INFO Finished job - poisson (45f843da-fde4-4fc1-8eda-5a6595fa6f53)
2025-04-04 14:49:45,326 INFO Starting job - plot_over_line (18182edf-93cd-4ee3-b8ec-2babbac189be)
2025-04-04 14:49:47,301 INFO Finished job - plot_over_line (18182edf-93cd-4ee3-b8ec-2babbac189be)
2025-04-04 14:49:47,302 INFO Starting job - substitute_macros (b94580b0-329b-4043-bc8e-04dfcc5e8443)
2025-04-04 14:49:48,572 INFO Finished job - substitute_macros (b94580

{'96c2a02e-82a1-4ec2-8c8f-03aaaaaa2199': {1: Response(output='/home/jovyan/preprocessing/square.msh', detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 '27e8a0e1-b612-4ab4-8dec-f13a2e92c152': {1: Response(output={'xdmf_file': '/home/jovyan/preprocessing/square.xdmf', 'h5_file': '/home/jovyan/preprocessing/square.h5'}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 '45f843da-fde4-4fc1-8eda-5a6595fa6f53': {1: Response(output={'numdofs': 357, 'pvd_file': '/home/jovyan/processing/poisson.pvd', 'vtu_file': '/home/jovyan/processing/poisson000000.vtu'}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 '18182edf-93cd-4ee3-b8ec-2babbac189be': {1: Response(output='/home/jovyan/postprocessing/plotoverline.csv', detour=None, additi